# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
iris

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [4]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [5]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [6]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [7]:
class OVR_SVM:
    def __init__(self, classes=1):
        '''
            num_classes : 클래스 개수
            clfs : classifier들
            pred_result : 예측 결과
        '''
        self.num_classes = classes
        self.clfs = []
        self.pred_result = []
    
    # 인코딩된 y_train과 X_train 학습 데이터로 SVM 분류 모델 훈련을 진행
    # y값을 one-hot encoding하여 예측하기 때문에 one-rest 방식으로 각각의 target class에 대한 예측을 진행할 수 있음
    # 0을 반환하는 경우에는 target class를 갖지 않는 것으로, 1을 반환하는 경우에는 target class를 갖는 것으로 판단하며 arg_max를 통해 계산
    def fit(self, X_train, y_train):
        encoded_y_train = pd.get_dummies(y_train)

        for i in range(self.num_classes):
            clf_machine = SVC(kernel='rbf', C=1, gamma=5)
            clf_machine.fit(X_train, encoded_y_train.iloc[:,i])    
            self.clfs.append(clf_machine)
    
    # 투표를 진행한 후에, 각 분류기에서 나온 결과 비교
    # row 개수만큼의 vote 형성
    def predict(self, X_test):
        row_count = X_test.shape[0]
        votes = np.zeros((row_count, self.num_classes), dtype=int)
        
        for _row in range(row_count):
            decision_func = list(map(lambda elem : elem.decision_function(X_test)[_row], self.clfs))
            self.pred_result.append(np.argmax(decision_func))
        
        # label 다시 변환
        self.pred_result = pd.DataFrame(self.pred_result).replace({0:'setosa', 1:'versicolor', 2:'virginica'})
        return self.pred_result
    
    # 정확도 확인
    def evaluate(self, y_test):
        print(f'Accuracy : {(accuracy_score(y_test, self.pred_result))}')

In [8]:
OVR = OVR_SVM(classes = 3)

# 데이터 fitting
OVR.fit(X_train, y_train)

# 결과 확인
pred_onerest = OVR.predict(X_test)
OVR.evaluate(y_test)

Accuracy : 0.8666666666666667
